In [24]:
import numpy as np
import pandas as pd
import re
import pickle


In [25]:
df = pd.read_excel('data.xlsx', index_col=0)

In [26]:
def remove_repeat_value(repeat_value: str, ls: list):
    # 将重复值替换为上一个出现的有用值
    columns = []
    for i in ls:
        if repeat_value in str(i):
            tmp_ls = ls.copy()
            tmp_ls.reverse()
            tmp_ls = tmp_ls[tmp_ls.index(i):]
            for x in tmp_ls:
                if repeat_value not in str(x):
                    columns.append(x)
                    break
        else:
            columns.append(i)
    return columns
df.columns = remove_repeat_value('Unnamed',list(df.columns))
df.index = ['课节'] + list(df.index[1:])

In [27]:
not_nan_index = []
reversed_index = list(df.index)
reversed_index.reverse()
len_index = len(df.index)
for n,i in enumerate(df.index.isna()):
    if i:
        for x in reversed_index[len_index-n:]:
            if str(x) != 'nan':
                not_nan_index.append(x)
                break
    else:
        not_nan_index.append(df.index[n])
df.index = not_nan_index

In [28]:
class Clazz:
    # 班级
    def __init__(self):
        self.class_name = None

        # 每天
        self.day_1 = []
        self.day_2 = []
        self.day_3 = []
        self.day_4 = []
        self.day_5 = []
        self.day_6 = []
        self.day_7 = []
        self.days = [self.day_1,self.day_2,self.day_3,self.day_4,self.day_5,self.day_6,self.day_7]

class Course:
    # 课程
    def __init__(self):
        self.course_name = ''
        self.id = 0
        # 开始周
        self.start_week = 0
        # 结束周
        self.endWeek = 0
        # 上课地点
        self.room = ''
        # 开始节
        self.start_node = 0
        # 结束
        self.step = 0
        # 认可老师
        self.teacher = 0
        self.day = 0

In [29]:
def parsing_course(text: str):
    # 解析课程
    # 电子线路辅助设计 令狐昌伟 [2-3,6-17]周 1-4节 汽车楼536嵌入式实训室 本部(东校区） 54
    # 课程名称    任课老师    周数    节数    地址
    text = text.split(' ')
    while True:
        if ']周' not in text[2]:
            text[1] = text[1] + text[2]
            text.pop(2)
        else:
            break
    weeks = []
    if text[2].count(',') != 0:
        text[2] = text[2].split(',')
        for i in text[2]:
            if '-' in i:
                i = i.split('-')
                weeks.append([i[0], i[1]])
            else:
                weeks.append([i])
    else:
        if '-' in text[2]:
            text[2] = text[2].split('-')
            weeks.append([text[2][0], text[2][1]])
        else:
            weeks.append([text[2]])

    weeks = list(map(lambda x: list(map(lambda x2: int(x2.replace('[', '').replace(']','').replace('周', '')), x)), weeks))

    courses = []
    for i in weeks:
        course = Course()
        # 开始周
        course.start_week = i[0]
        # 结束周
        if len(i) == 1:
            course.endWeek = i[0]
        else:
            course.endWeek = i[1]
        # 课程名称
        course.course_name = text[0]
        # 任课老师
        course.teacher = text[1]
        # 开始节
        course.start_node = int(text[3].split('-')[0])
        # 结束节
        course.step = int(text[3].split('-')[1].replace('节', ''))
        course.step = course.step - course.start_node
        # room
        course.room = text[4]
        courses.append(course)
    return courses




In [30]:
# 创建班级类
classes = dict()
for i in list(df.iterrows())[1:]:
    clazzname = re.match('.*\(', i[0]).group()[:-1]
    if not classes.get(clazzname, False):
        classes[clazzname] = Clazz()
    classes[clazzname].class_name =clazzname

    for x in set(list(i[1]['星期一'])):
        if str(x) != 'nan':
            classes[clazzname].day_1 += parsing_course(x)
    for x in set(list(i[1]['星期二'])):
        if str(x) != 'nan':
            classes[clazzname].day_2 += parsing_course(x)
    for x in set(list(i[1]['星期三'])):
        if str(x) != 'nan':
            classes[clazzname].day_3 += parsing_course(x)
    for x in set(list(i[1]['星期四'])):
        if str(x) != 'nan':
            classes[clazzname].day_4 += parsing_course(x)
    for x in set(list(i[1]['星期五'])):
        if str(x) != 'nan':
            classes[clazzname].day_5 += parsing_course(x)
    for x in set(list(i[1]['星期六'])):
        if str(x) != 'nan':
            classes[clazzname].day_6 += parsing_course(x)
    for x in set(list(i[1]['星期日'])):
        if str(x) != 'nan':
            classes[clazzname].day_7 += parsing_course(x)


In [31]:
with open('classes', 'wb') as f:
    f.write(pickle.dumps(classes))